실제 음성 변환 모델의 구현

In [ ]:
# 1) 라이브러리 설치
!pip install -q librosa soundfile

# 2) 파일 업로드 (source.wav 라는 이름의 .wav 파일을 준비하세요)
from google.colab import files
print("▶ 5–10초 분량의 source.wav 파일을 업로드하세요")
uploaded = files.upload()
source_file = next(f for f in uploaded if f.lower().endswith(".wav"))
print("업로드된 파일:", source_file)

# 3) 음성 로드
import librosa, soundfile as sf
y, sr = librosa.load(source_file, sr=None)  # 원본 그대로의 샘플링레이트 유지

# 4) 피치 쉬프트 (n_steps = +4 하면 반음 4개 올림, 음색이 높아짐)
n_steps = 4  # 원하는 만큼 조절 (음색을 낮추려면 음수가능, e.g. -4)
y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

# 5) (선택) 속도 변경도 해보고 싶다면 time_stretch 사용
rate = 1.2  # 1보다 크면 빨라짐, 작으면 느려짐
y_speed = librosa.effects.time_stretch(y_shifted, rate=rate)

# 6) 결과 저장
output_file = "converted_voice.wav"
sf.write(output_file, y_shifted, sr)
#sf.write(output_file, y_speed, sr) #속도 변경도 할 경우 위 코드는 주석처리 해야한다.
print("저장된 변환 음성:", output_file)

# 7) Colab 상에서 재생
from IPython.display import Audio, display
display(Audio(output_file, rate=sr))

다양한 음성변환

In [ ]:
# 1) 의존성 설치
!pip install -q librosa soundfile numpy

# 2) .wav 파일 업로드
from google.colab import files
print("5~10초 분량의 source.wav 파일을 업로드하세요")
uploaded = files.upload()
src = next(f for f in uploaded.keys() if f.lower().endswith(".wav"))
print("사용할 파일:", src)

# 3) 라이브러리 임포트
import numpy as np
import librosa, soundfile as sf
from IPython.display import Audio, display

# 4) 원본 로드
y, sr = librosa.load(src, sr=None)
print(f"Loaded {src}, samplerate = {sr}, duration = {y.shape[0]/sr:.2f}s")

# 5) 효과 함수 정의

def pitch_shift(y, sr, n_steps):  #pitch 주파수 이동
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

def time_stretch(y, rate):  #sampling rate 변환 -> 속도 조절
    return librosa.effects.time_stretch(y, rate=rate)

def echo(y, sr, delay=0.3, attenuation=0.6): #반향 -> echo(매아리) 소리가 장애물에 부딪쳐서 돌아온는 소리다. 즉 원본 소리와 크기가 줄어든 소리가 계속 흐른다.
    nd = int(delay * sr)
    y_echo = np.zeros_like(y)
    y_echo[nd:] = y[:-nd] * attenuation
    return y + y_echo

def reverb(y, sr, reverb_scale=0.5): #잔향 -> 여러 개의 path가 적용된다. 예를 들어 뒤에 스피커가 있으면 이 스피커에서 시작된 소리가 내 귀로 들어올 때까지의 경로는 여러 개이다. 각 경로마다 소리의 길이가 다르다. 넓은 공간에서 소리를 듣는 것처럼 느끼게 된다.
    # 짧은 지수 감쇠 임펄스 응답 생성 (30ms)
    ir_len = int(0.03 * sr)
    ir = np.logspace(0, -3, ir_len)
    y_rev = np.convolve(y, ir)[:len(y)]
    return (1 - reverb_scale) * y + reverb_scale * y_rev

def distortion(y, threshold=0.3): #원래 음을 깨지도록 함
    return np.clip(y, -threshold, threshold)

# 6) 각 이펙트 적용 & 저장 & 재생

effects = {
    "pitch_up_4":  pitch_shift(y, sr, +4),
    "pitch_down_4":pitch_shift(y, sr, -4),
    "slow_0.8":    time_stretch(y, 0.8),
    "fast_1.2":    time_stretch(y, 1.2),
    "echo":        echo(y, sr, delay=0.25, attenuation=0.5),
    "reverb":      reverb(y, sr, reverb_scale=0.4),
    "distortion":  distortion(y, threshold=0.2),
}

for name, y_eff in effects.items():
    # 길이 맞추기 (time_stretch만 길이가 다름)
    if len(y_eff) > len(y):
        y_eff = y_eff[:len(y)]
    else:
        y_eff = np.pad(y_eff, (0, len(y) - len(y_eff)))
    fn = f"out_{name}.wav"
    sf.write(fn, y_eff, sr)
    print(f"▶ {name} saved to {fn}")
    display(name, Audio(fn, rate=sr))

voice cloning

음성 복제 즉 목소리 흉내이다.



In [ ]:
# @title Voice Cloning

# 1. 필요한 라이브러리 설치
print("Installing necessary libraries...")
!pip install -q transformers accelerate scipy librosa soundfile numpy


In [ ]:

# 2. 모듈 임포트
import torch
from transformers import AutoProcessor, AutoModel
from scipy.io.wavfile import write as write_wav
import IPython.display as ipd
import librosa
import numpy as np
import os
from google.colab import files

# 3. 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 4. Bark 모델 로드 (데모용, 실제 사용 시 TARGET_BARK 는 이미 준비된 16kHz 파일)
print("Loading Bark model and processor...")
processor = AutoProcessor.from_pretrained("suno/bark-small")
model = AutoModel.from_pretrained("suno/bark-small").to(device)
print("Bark model loaded successfully!")

# 5. Bark 합성 함수 정의 (필요시)
def generate_bark_audio(text_input, speaker_preset="v2/en_speaker_6", filename="bark_output.wav"):
    inputs = processor(text_input, voice_preset=speaker_preset)
    for k, v in inputs.items():
        if isinstance(v, torch.Tensor):
            inputs[k] = v.to(device)
    with torch.no_grad():
        speech_output = model.generate(**inputs, do_sample=True, fine_tuned_audio_output=True)
    # Bark는 24kHz로 생성하므로, 이후 16kHz로 리샘플할 것
    tmp_wav = "tmp_bark.wav"
    write_wav(tmp_wav, 24000, speech_output.cpu().numpy().squeeze())
    # 16kHz 리샘플
    y, _ = librosa.load(tmp_wav, sr=16000)
    write_wav(filename, 16000, (y * 32767).astype(np.int16))
    os.remove(tmp_wav)
    return filename

# 6. Voice Cloning: 피치 매칭 방식 (샘플링레이트 16kHz)
print("▶ Upload your reference voice AND your target TTS .wav files (2 files, both 16kHz).")
uploaded = files.upload()
wav_files = [f for f in uploaded if f.lower().endswith(".wav")]
if len(wav_files) != 2:
    raise RuntimeError("두 개의 16kHz .wav 파일(참조 음성, TTS 결과)을 업로드해야 합니다.")
REF_VOICE, TARGET_BARK = wav_files #REF_VOICE에서 특징 뽑아서 TARGET으로 갔다.
                                   #REF_VOICE의 Pitch만 뽑아서 TARGET으로 넘긴 것이다.
                                   #이때 TARGET도 18일차에 녹음한 나의 목소리이지만
                                   #REF_VOICE의 Pitch를 강제로 TARGET에다가 넣은 것이다.

print(f"Reference voice: {REF_VOICE}")
print(f"Target TTS output: {TARGET_BARK}")

# 7. 파일 로드 및 피치 매칭 (sr=16000)
y_ref, sr_ref = librosa.load(REF_VOICE, sr=16000)
y_tgt, sr_tgt = librosa.load(TARGET_BARK, sr=16000)

# 피치(F0) 추정
f0_ref = librosa.yin(y_ref, fmin=50, fmax=500, sr=sr_ref)
f0_tgt = librosa.yin(y_tgt, fmin=50, fmax=500, sr=sr_tgt)
avg_ref = np.nanmean(f0_ref)
avg_tgt = np.nanmean(f0_tgt)
n_steps = 12 * np.log2(avg_ref / avg_tgt)

# 피치 시프트 (모두 키워드 인자로 전달)
y_shifted = librosa.effects.pitch_shift(
    y=y_tgt,
    sr=sr_tgt,
    n_steps=n_steps
)

# 8. 결과 저장 및 재생 (16kHz)
CLONED_OUT = "bark_cloned_16k.wav"
write_wav(CLONED_OUT, 16000, (y_shifted * 32767).astype(np.int16))
print(f"✅ Cloned voice saved to {CLONED_OUT}")
ipd.display(ipd.Audio(CLONED_OUT, rate=16000))

In [ ]:
# @title Voice Cloning (Pitch + Speed Clone, 16 kHz)

# 1. 필요한 라이브러리 설치
!pip install -q transformers accelerate scipy librosa soundfile numpy

# 2. 모듈 임포트
import torch
from transformers import AutoProcessor, AutoModel
from scipy.io.wavfile import write as write_wav
import IPython.display as ipd
import librosa
import numpy as np
import os
from google.colab import files

# 3. 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 4. Bark 모델 로드 (데모용)
processor = AutoProcessor.from_pretrained("suno/bark-small")
model = AutoModel.from_pretrained("suno/bark-small").to(device)

# 5. Bark 합성 함수 (필요 시)
def generate_bark_audio(text_input, speaker_preset="v2/en_speaker_6", filename="bark_output.wav"):
    inputs = processor(text_input, voice_preset=speaker_preset)
    for k, v in inputs.items():
        if isinstance(v, torch.Tensor):
            inputs[k] = v.to(device)
    with torch.no_grad():
        speech_output = model.generate(**inputs, do_sample=True, fine_tuned_audio_output=True)
    tmp = "tmp_bark.wav"
    write_wav(tmp, 24000, speech_output.cpu().numpy().squeeze())
    y, _ = librosa.load(tmp, sr=16000)
    write_wav(filename, 16000, (y * 32767).astype(np.int16))
    os.remove(tmp)
    return filename

# 6. Voice Cloning: 피치와 속도 클론
print("Upload two 16kHz WAV files: reference voice and target TTS output.")
uploaded = files.upload()
wav_files = [f for f in uploaded if f.lower().endswith(".wav")]
if len(wav_files) != 2:
    raise RuntimeError("두 개의 16kHz WAV 파일을 업로드해야 합니다.")
REF_VOICE, TARGET_BARK = wav_files

# 7. 파일 로드
sr = 16000
y_ref, _ = librosa.load(REF_VOICE, sr=sr) #화자의 특징을 뽑기 위함
y_tgt, _ = librosa.load(TARGET_BARK, sr=sr) #화자의 특징을 적용하기 위함

# 8. 속도 매칭
dur_ref = len(y_ref) / sr
dur_tgt = len(y_tgt) / sr
rate = dur_tgt / dur_ref
y_rate = librosa.effects.time_stretch(y=y_tgt, rate=rate)

# 9. 피치 매칭
f0_ref = librosa.yin(y=y_ref, fmin=50, fmax=500, sr=sr)
f0_rate = librosa.yin(y=y_rate, fmin=50, fmax=500, sr=sr)
n_steps = 12 * np.log2(np.nanmean(f0_ref) / np.nanmean(f0_rate))
y_shift = librosa.effects.pitch_shift(y=y_rate, sr=sr, n_steps=n_steps)

# 10. 길이 맞추기
if len(y_shift) > len(y_ref):
    y_final = y_shift[:len(y_ref)]
else:
    y_final = np.pad(y_shift, (0, len(y_ref) - len(y_shift)))

# 11. 저장 및 재생
OUT = "voice_cloned_full.wav"
write_wav(OUT, sr, (y_final * 32767).astype(np.int16))
print("Cloned voice saved to", OUT)
ipd.display(ipd.Audio(REF_VOICE, rate=sr))
ipd.display(ipd.Audio(OUT, rate=sr))

In [ ]:
# @title Voice Cloning (Reference + Hardcoded Text, 16 kHz)

# 1. 필요한 라이브러리 설치
!pip install -q transformers accelerate scipy librosa soundfile numpy

# 2. 모듈 임포트
import torch
from transformers import AutoProcessor, AutoModel
from scipy.io.wavfile import write as write_wav
import IPython.display as ipd
import librosa
import numpy as np
import os
from google.colab import files

# 3. 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 4. Bark 모델 로드 (24 kHz→16 kHz 용)
processor = AutoProcessor.from_pretrained("suno/bark-small")
model     = AutoModel.from_pretrained("suno/bark-small").to(device)

# 5. Hardcoded 텍스트
SYNTH_TEXT = "안녕하세요. 이 목소리로 새로운 문장을 합성합니다."

# 6. Bark TTS 생성 함수 (24→16 kHz)
def synthesize_bark(text, out16k="tts_16k.wav"):
    inputs = processor(text, voice_preset="v2/ko_speaker_6")
    for k,v in inputs.items():
        if isinstance(v, torch.Tensor):
            inputs[k] = v.to(device)
    with torch.no_grad():
        speech = model.generate(
            **inputs,
            do_sample=True,
            fine_tuned_audio_output=True
        )
    # 24 kHz 저장 → 16 kHz 리샘플
    tmp = "tmp24k.wav"
    write_wav(tmp, 24000, speech.cpu().numpy().squeeze())
    y, _ = librosa.load(tmp, sr=16000)
    write_wav(out16k, 16000, (y * 32767).astype(np.int16))
    os.remove(tmp)
    return out16k

# 7. 참조 음성 업로드 (16 kHz WAV 한 개)
print("▶ Upload ONE reference voice (16 kHz WAV):")
uploaded = files.upload()
wav_files = [f for f in uploaded if f.lower().endswith(".wav")]
if len(wav_files) != 1:
    raise RuntimeError("16 kHz WAV 파일을 한 개만 업로드해주세요.")
REF_VOICE = wav_files[0]
print("Reference voice:", REF_VOICE)

# 8. Bark TTS 생성
print("\n▶ Generating TTS from hardcoded text …")
TTS_WAV = synthesize_bark(SYNTH_TEXT)
print("Generated TTS file:", TTS_WAV)
ipd.display(ipd.Audio(TTS_WAV, rate=16000))

# 9. 파일 로드 (sr=16 kHz)
sr = 16000
y_ref, _ = librosa.load(REF_VOICE, sr=sr)
y_tgt, _ = librosa.load(TTS_WAV,  sr=sr)

# 10. 속도 매칭
dur_ref = len(y_ref) / sr
dur_tgt = len(y_tgt) / sr
rate    = dur_tgt / dur_ref
y_rate  = librosa.effects.time_stretch(y=y_tgt, rate=rate)

# 11. 피치 매칭
f0_ref  = librosa.yin(y=y_ref,  fmin=50, fmax=500, sr=sr)
f0_rt   = librosa.yin(y=y_rate, fmin=50, fmax=500, sr=sr)
n_steps = 12 * np.log2(np.nanmean(f0_ref) / np.nanmean(f0_rt))
y_clone = librosa.effects.pitch_shift(y=y_rate, sr=sr, n_steps=n_steps)

# 12. 길이 맞추기 (참조 음성 기준)
if len(y_clone) > len(y_ref):
    y_final = y_clone[:len(y_ref)]
else:
    y_final = np.pad(y_clone, (0, len(y_ref) - len(y_clone)))

# 13. 저장 및 재생
OUT = "voice_cloned_full.wav"
write_wav(OUT, sr, (y_final * 32767).astype(np.int16))
print("\n Cloned voice saved:", OUT)
print("▶ Reference voice:")
ipd.display(ipd.Audio(REF_VOICE, rate=sr))
print("▶ Cloned output:")
ipd.display(ipd.Audio(OUT, rate=sr))

Bark실습

In [ ]:
# @title Bark TTS 모델 설치 및 데모 실행 (단일 스크립트)

# 1. 필요한 라이브러리 설치
# transformers: Bark 모델 로드 및 사용
# accelerate: 모델 로딩 및 실행 속도 최적화 (GPU 사용 시)
# scipy: 오디오 파일 저장

#이 코드를 실행하기 위해서는 cpu모드로 전환해야 한다.
print("Installing necessary libraries...")
!pip install -q transformers accelerate scipy

# 2. 필요한 모듈 임포트
import torch
from transformers import AutoProcessor, AutoModel
from scipy.io.wavfile import write as write_wav
import IPython.display as ipd # Colab에서 오디오 재생을 위해

# 3. GPU 사용 가능 여부 확인 및 디바이스 설정
# Colab에서 '런타임' -> '런타임 유형 변경' -> 'T4 GPU' 선택 권장
if torch.cuda.is_available():
    device = "cuda"
    print("CUDA (GPU) is available! Using GPU for Bark.")
else:
    device = "cpu"
    print("CUDA (GPU) is not available. Using CPU. This may be slow.")

# 4. Bark 모델 및 프로세서 로드
# suno/bark-small은 더 가벼운 버전이며, suno/bark는 풀 사이즈 모델입니다.
# 처음 실행 시 모델을 다운로드하므로 시간이 걸릴 수 있습니다.
print("Loading Bark model and processor...")
processor = AutoProcessor.from_pretrained("suno/bark-small") # 또는 "suno/bark"
model = AutoModel.from_pretrained("suno/bark-small").to(device) # 또는 "suno/bark"
print("Bark model loaded successfully!")

# 5. 텍스트를 음성으로 변환하는 함수 정의
def generate_bark_audio(text_input, speaker_preset="v2/en_speaker_6", filename="bark_output.wav"):
    """
    Bark 모델을 사용하여 텍스트 프롬프트를 오디오로 변환합니다.

    Args:
        text_input (str): 음성으로 변환할 텍스트.
                           감성이나 어조를 표현하는 텍스트도 포함할 수 있습니다.
                           예: "안녕하세요. [행복한 표정] 오늘 날씨가 정말 좋네요!"
        speaker_preset (str): 사용할 스피커 프리셋 (예: "v2/en_speaker_6", "v2/ko_speaker_6").
                              다양한 프리셋이 있으며, 각각 다른 목소리와 억양을 가집니다.
                              한국어는 'v2/ko_speaker_X' 형식으로 시도할 수 있습니다.
        filename (str): 생성된 오디오 파일을 저장할 이름.
    """
    print(f"\nGenerating audio for text: '{text_input}'")
    print(f"Using speaker preset: '{speaker_preset}'")

    # 입력 텍스트를 Bark 모델에 맞는 형태로 인코딩
    inputs = processor(text_input, voice_preset=speaker_preset).to(device)

    # 오디오 생성
    with torch.no_grad():
        speech_output = model.generate(**inputs, do_sample=True, fine_tuned_audio_output=True)

    # 생성된 오디오를 WAV 파일로 저장
    # Bark 모델의 기본 샘플링 레이트는 24000 Hz입니다.
    bark_sample_rate = 24000
    write_wav(filename, bark_sample_rate, speech_output.cpu().numpy().squeeze())
    print(f"Audio saved to '{filename}' with sample rate {bark_sample_rate} Hz.")

    # Colab에서 오디오 재생
    print("Playing audio...")
    return ipd.Audio(filename)

# 6. Bark TTS 데모 실행 예시

# 예시 1: 영어 (기본 스피커 프리셋)
english_text = "Hello everyone. [laughs] I hope you are having a wonderful day today!"
generate_bark_audio(english_text, filename="english_demo.wav")

# 예시 2: 한국어
# 한국어 스피커 프리셋은 'v2/ko_speaker_X' 형식으로 시도할 수 있습니다.
# X는 0부터 9까지의 숫자일 수 있으며, 각각 다른 목소리를 가집니다.
korean_text_1 = "안녕하세요. 오늘 날씨가 정말 좋네요. 즐거운 하루 보내세요!"
generate_bark_audio(korean_text_1, speaker_preset="v2/ko_speaker_6", filename="korean_demo_1.wav")

korean_text_2 = "중요한 공지입니다. 모든 직원분들은 회의실로 모여주시기 바랍니다. [진지한 어조]"
generate_bark_audio(korean_text_2, speaker_preset="v2/ko_speaker_9", filename="korean_demo_2.wav")

korean_text_3 = "정말 믿을 수가 없어! [놀란 어조] 이게 사실이라고?"
generate_bark_audio(korean_text_3, speaker_preset="v2/ko_speaker_0", filename="korean_demo_3.wav")

# 참고: Bark는 텍스트 프롬프트 안에 [laughs], [sighs], [music], [narrator] 등
# 다양한 지시어를 포함하여 음향 효과나 어조를 유도할 수 있습니다.
# 하지만 모든 지시어가 항상 완벽하게 작동하는 것은 아니며, 한국어의 경우 더욱 실험이 필요할 수 있습니다.